In [20]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [21]:
!pip install langchain_core langchain_community langchain_openai langchain_chroma pypdf -q


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [22]:
from langchain_community.document_loaders import PyPDFLoader

from langchain_text_splitters import TokenTextSplitter

from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.runnables import RunnablePassthrough

from langchain_openai import (ChatOpenAI,
                              OpenAIEmbeddings)

from langchain_chroma.vectorstores import Chroma

In [23]:
loader = PyPDFLoader("IntroToAICourseNotes.pdf")

docs_list = loader.load()
# docs_list

In [ ]:
token_splitter = TokenTextSplitter(encoding_name="cl100k_base",
                                   chunk_size=200,
                                   chunk_overlap=40)

docs_list_tokens_split = token_splitter.split_documents(docs_list)

embedding = OpenAIEmbeddings(model='text-embedding-3-small')

vectorstore = Chroma.from_documents(documents = docs_list_tokens_split,
                                    embedding = embedding,
                                    persist_directory = "./intro-to-ai")

In [31]:
PROMPT_RETRIEVING_S = '''You will receive a question from a student taking the Intro to AI course.
Answer the question using only the provided context.'''

PROMPT_TEMPLATE_RETRIEVING_H = '''This is the question:
{question}

This is the context:
{context}'''

prompt_retrieving_s = SystemMessage(PROMPT_RETRIEVING_S)
prompt_template_retrieving_h = HumanMessagePromptTemplate.from_template(PROMPT_TEMPLATE_RETRIEVING_H)

chat_prompt_template_retrieving = ChatPromptTemplate([prompt_retrieving_s,
                                                      prompt_template_retrieving_h])


In [32]:
chat = ChatOpenAI(model_name = "gpt-4o",
                  temperature = 0)

str_output_parser = StrOutputParser()

chain = ({'context': retriever,
          'question': RunnablePassthrough()}
         | chat_prompt_template_retrieving
         | chat
         | str_output_parser
        )

response = chain.stream('Talk about the history of AI. Return the answer in bullet points.')

for chunk in response:
    print(chunk, end = "")

- **1950**: Alan Turing publishes a paper asking, "Can machines think?" and introduces the Turing Test, setting a practical criterion for evaluating machine intelligence.
- **1956**: The Dartmouth Conference marks the formal start of AI as a field of study, coining the term "artificial intelligence."
- **1960s and 70s**: AI Winter occurs due to limited technology and data availability, leading to reduced funding and interest.
- **1997**: IBM’s Deep Blue defeats world chess champion Garry Kasparov, reigniting interest in AI.
- **Late 1990s and Early 2000s**: A surge in computer power and the rapid expansion of the Internet provide resources for advanced AI research.
- **2006**: Geoffrey Hinton’s paper on deep learning revives interest in neural networks, introducing techniques that mimic the human brain's functions.